# Replication Analysis: Income Moderates Age-Related Financial Patience
### A Python implementation of the findings from Wan et al. (2024), *Psychology and Aging*

---

### Objective

This notebook provides a complete, reproducible Python workflow for the analyses presented in the following publication:

> Wan, H., Myerson, J., Green, L., Strube, M. J., & Hale, S. (2024). Age-related differences in delay discounting: Income matters. *Psychology and Aging*. Advance online publication. https://doi.org/10.1037/pag0000818

The original analysis was conducted in R, and this notebook serves to replicate the findings using the Python data science ecosystem. The project's goal is to test the **"buffering hypothesis"**—the theory posits that the greater emotional stability of older adults buffers them against the financial stress of scarcity, leading to more patient financial decisions compared to their younger, low-income counterparts.

### Technical Workflow

The analysis is structured in five sequential stages:

1.  **Setup & Data Import**: Load required Python libraries (`pandas`, `statsmodels`, `pymc`, etc.) and the raw dataset.
2.  **Data Processing & Validation**: Clean the data, create analysis-ready dataframes, and perform data quality and reliability checks.
3.  **Hypothesis Testing 1 (Age Effects)**: Replicate the Bayesian multilevel models from Table 2 of the paper, testing the effect of age on financial patience within each income group.
4.  **Hypothesis Testing 2 (Income Effects)**: Replicate the models from Table 3, testing the effect of income within each age group.
5.  **Composite Score Analysis**: Create a composite z-score to perform final correlation tests and quantify the magnitude of the key interaction effect.

The data and original study materials are publicly available on the Open Science Framework at [https://osf.io/um68t/](https://osf.io/um68t/).

In [ ]:
# --- Environment Setup ---
#
# This cell installs the required Python packages for the analysis.
# Uncomment and run this cell only if you are setting up a new environment.

# import sys
# !{sys.executable} -m pip install pandas numpy openpyxl scipy statsmodels pymc arviz scikit-learn

In [ ]:
# --- 1. SETUP: IMPORTS, FUNCTIONS, AND DATA PROCESSING ---

# --- 1.1 Load Libraries ---

# Core data science libraries
import pandas as pd
import numpy as np
import warnings

# Statistical modeling and analysis
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pymc as pm
import arviz as az
from scipy.stats import pearsonr

# Suppress warnings for a cleaner final report
warnings.filterwarnings('ignore')
# Set pandas display options for consistent formatting
pd.options.display.float_format = '{:.3f}'.format


# --- 1.2 Custom Bayesian Modeling Functions ---

def fit_bayesian_glmm(formula: str, data: pd.DataFrame, family: str = 'binomial', progressbar: bool = False):
    """
    Fits a Bayesian Generalized Linear Mixed-Effects Model using PyMC.

    This function is designed to replicate the brms models from the R analysis,
    including a random intercept for participant ID.

    Args:
        formula (str): A statsmodels-style formula (e.g., "outcome ~ predictor").
        data (pd.DataFrame): The dataframe containing the model variables.
        family (str): The likelihood distribution ('binomial' or 'beta').
        progressbar (bool): Whether to display the MCMC sampling progress bar.

    Returns:
        arviz.InferenceData: The fitted model object containing posterior samples.
    """
    outcome, predictors = formula.split('~')
    outcome = outcome.strip()
    
    # Use patsy to parse the formula and create design matrices
    y, X = pd.factorize(data['ID'])
    coords = {"obs_id": np.arange(data.shape[0]), "participant": X}
    participant_idx = y

    with pm.Model(coords=coords) as model:
        # --- Priors ---
        # Priors are chosen to be weakly informative, matching the R brms setup.
        intercept = pm.Normal("Intercept", mu=0, sigma=10)
        # Assumes a single slope for simplicity in this helper function
        slope = pm.Normal(predictors.strip(), mu=0, sigma=2.5) 
        # Random intercepts for each participant
        sd_participant = pm.HalfCauchy("sd_participant", beta=2.5)
        z_participant = pm.Normal("z_participant", mu=0, sigma=1, dims="participant")
        participant_effect = z_participant * sd_participant
        
        # --- Linear Model ---
        eta = (intercept +  slope * data[predictors.strip()] + participant_effect[participant_idx])

        # --- Likelihood ---
        if family == 'binomial':
            p = pm.math.invlogit(eta)
            pm.Binomial(
                outcome,
                n=data['trials'].values,
                p=p,
                observed=data[outcome].values,
                dims="obs_id"
            )
        elif family == 'beta':
            # For Beta regression, the precision parameter (nu) also needs a prior
            nu = pm.HalfCauchy("nu", beta=10) 
            mu = pm.math.invlogit(eta)
            pm.Beta(
                outcome,
                mu=mu,
                nu=nu, 
                observed=data[outcome].values,
                dims="obs_id"
            )

        # --- MCMC Sampling ---
        idata = pm.sample(draws=2000, tune=2000, chains=4, cores=4, target_accept=0.95, progressbar=progressbar)
    return idata

def summarize_pymc_model(idata, var_names=None):
    """
    Creates a clean summary table from a PyMC InferenceData object.

    Args:
        idata (arviz.InferenceData): A fitted PyMC model object.
        var_names (list, optional): List of variable names to include in the summary.

    Returns:
        pd.DataFrame: A formatted summary table including mean, HDI, and pd.
    """
    summary_df = az.summary(idata, hdi_prob=0.95, kind='stats', var_names=var_names)
    
    # Calculate Probability of Direction (pd)
    posterior = az.extract(idata, var_names=var_names)
    pd_values = {}
    for var in posterior.data_vars:
        samples = posterior[var].values.flatten()
        pd_val = np.mean(samples > 0)
        pd_val = max(pd_val, 1 - pd_val) # Ensure pd is always >= 0.5
        pd_values[var] = pd_val
        
    summary_df['pd'] = summary_df.index.map(pd_values)
    summary_df['significance'] = np.where(summary_df['pd'] >= 0.975, '*', ' ')
    summary_df = summary_df[['mean', 'hdi_2.5%', 'hdi_97.5%', 'pd', 'significance']]
    summary_df.columns = ['Median', 'CI_lower', 'CI_upper', 'pd', 'sig']
    
    return summary_df


# --- 1.3 Load and Process Data ---

# Load raw data from the two sheets in the Excel file
raw_mcq_df = pd.read_excel("Data_AgeIncome.xlsx", sheet_name="MCQ")
raw_adj_amt_df = pd.read_excel("Data_AgeIncome.xlsx", sheet_name="Adj-Amt")

# Create a lookup table for MCQ item parameters - a much cleaner approach
mcq_item_params = pd.DataFrame({
    'Q_ID': range(1, 28),
    'k': [0.00016, 0.006, 0.006, 0.25, 0.041, 0.0004, 0.1, 0.1, 0.00016, 0.006, 
          0.25, 0.001, 0.00016, 0.041, 0.0025, 0.0025, 0.0004, 0.016, 0.1, 0.0004, 
          0.016, 0.0025, 0.041, 0.001, 0.016, 0.001, 0.25],
    'Amount': ["$55", "$80", "$30", "$80", "$30", "$55", "$30", "$55", "$80", 
               "$55", "$30", "$80", "$30", "$55", "$80", "$55", "$80", "$30", 
               "$80", "$30", "$55", "$30", "$80", "$55", "$80", "$30", "$55"]
})

# Merge the parameters with the raw MCQ data
mcq_df_with_params = pd.merge(raw_mcq_df, mcq_item_params, on='Q_ID')
mcq_df_with_params['Amount'] = pd.Categorical(
    mcq_df_with_params['Amount'], 
    categories=["$80", "$55", "$30"], ordered=True
)

# Calculate participant-level summaries for each behavioral task
adj_amt_participant_summary = raw_adj_amt_df[raw_adj_amt_df['Delay'] != 730].groupby(['ID', 'Amount']).apply(
    lambda g: pd.Series({'auc': np.trapz(g['RSV'], g['Delay'] / g['Delay'].max())})
).reset_index()

mcq_participant_summary = mcq_df_with_params.groupby(['ID', 'Amount'], observed=True).agg(
    num_delayed_choices=('Choice', 'sum')
).reset_index()

# Create master demographics dataframe and join to behavioral summaries
demographics_df = raw_adj_amt_df[['ID', 'Group', 'Age', 'Gender', 'Education', 'HADS']].drop_duplicates()
demographics_df['age_group'] = np.where(demographics_df['Group'].isin([1, 2]), 0, 1)    # 0=Younger
demographics_df['income_group'] = np.where(demographics_df['Group'].isin([1, 3]), 0, 1) # 0=Lower
demographics_df['education_group'] = np.where(demographics_df['Education'] <= 3, 0, 1)
demographics_df['hads_score'] = pd.to_numeric(demographics_df['HADS'], errors='coerce')

mcq_analysis_df = pd.merge(mcq_participant_summary, demographics_df, on='ID')
adj_amt_analysis_df = pd.merge(adj_amt_participant_summary, demographics_df, on='ID')

# Create model-specific dataframes with centered variables
def add_centered_predictors(df, cols):
    df_copy = df.copy()
    for col in cols:
        df_copy[f'{col}_c'] = df_copy[col] - df_copy[col].mean()
    return df_copy

mcq_model1_df = mcq_analysis_df[mcq_analysis_df['Amount'].isin(["$30", "$80"])]
mcq_model1_df = add_centered_predictors(mcq_model1_df, ['age_group', 'income_group'])
mcq_model1_df['trials'] = 9 # Number of choices per block

adj_amt_model1_df = adj_amt_analysis_df[adj_amt_analysis_df['Amount'].isin([30, 80])]
adj_amt_model1_df = add_centered_predictors(adj_amt_model1_df, ['age_group', 'income_group'])

# Create the composite z-score dataframe
mcq_z = mcq_analysis_df[mcq_analysis_df['Amount'].isin(["$30", "$80"])].sort_values(['ID', 'Amount'], ascending=[True, False])
adj_z = adj_amt_analysis_df[adj_amt_analysis_df['Amount'].isin([30, 80])]
composite_score_df = mcq_z[['ID', 'age_group', 'income_group', 'Age', 'num_delayed_choices']].copy()
composite_score_df['auc'] = adj_z['auc'].values
composite_score_df['mcq_z'] = (composite_score_df['num_delayed_choices'] - composite_score_df['num_delayed_choices'].mean()) / composite_score_df['num_delayed_choices'].std()
composite_score_df['adj_amt_z'] = (composite_score_df['auc'] - composite_score_df['auc'].mean()) / composite_score_df['auc'].std()
composite_score_df['mean_z_score'] = composite_score_df[['mcq_z', 'adj_amt_z']].mean(axis=1)
composite_score_df = composite_score_df.groupby('ID').mean().reset_index() # Average z-score per participant
composite_score_df = add_centered_predictors(composite_score_df, ['age_group', 'income_group'])
composite_score_df['age_c'] = (composite_score_df['Age'] - composite_score_df['Age'].mean()) / composite_score_df['Age'].std()

## 2. Data Quality & Reliability Checks

Before testing the primary hypotheses, this section replicates the initial analyses from the paper that establish the quality and internal consistency of the behavioral data. These checks confirm two key points:

1.  **Validity Check**: We fit established mathematical models of choice behavior (a logistic growth function for MCQ data and a hyperboloid function for Adj-Amt data) to the group-level data. R-squared values indicate that participants' choices were systematic and not random, conforming to theoretical expectations.
2.  **Reliability Check**: We assess the internal consistency of our measures by correlating participants' scores across the different reward amounts within each task. High positive correlations demonstrate that the tasks are reliable; for example, a participant who was impatient for a small reward was also likely to be impatient for a large one.

In [ ]:
# --- 2.1 Group-Level Model Fit Assessment ---

# Import the specific functions required for this cell's analysis
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# The following analyses assess how well standard mathematical models of choice
# describe the aggregated data from each of the four participant groups.
# High R-squared values indicate that the data is well-behaved and systematic.

# Define the mathematical functions for curve fitting
def logistic_growth(log_k, a, r):
    """Logistic growth function used to model MCQ choice proportions."""
    return 1 / (1 + np.exp(-(log_k - a) * r))

def hyperboloid(delay, k, s):
    """Hyperboloid function used to model delay discounting."""
    return 1 / (1 + k * delay)**s

# --- Create aggregated dataframes for group-level analysis ---
group_labels = {
    1: "Younger, Lower-Income", 2: "Younger, Higher-Income",
    3: "Older, Lower-Income", 4: "Older, Higher-Income"
}
mcq_df_with_params['group_label'] = mcq_df_with_params['Group'].map(group_labels)
raw_adj_amt_df['group_label'] = raw_adj_amt_df['Group'].map(group_labels)

mcq_group_summary = mcq_df_with_params.groupby(['group_label', 'Amount', 'k'], observed=True).agg(
    prop_delayed=('Choice', 'mean')
).reset_index()
mcq_group_summary['log_k'] = np.log(mcq_group_summary['k'])

adj_amt_group_summary = raw_adj_amt_df.groupby(['group_label', 'Amount', 'Delay']).agg(
    mean_rsv=('RSV', 'mean')
).reset_index()

# --- Fit models and calculate R-squared ---
print("--- MCQ: R-squared for Group-Level Logistic Growth Fits ---")
def calculate_r2(df, x_col, y_col, model_func, p0):
    """Helper function to fit a curve and return R-squared."""
    params, _ = curve_fit(model_func, df[x_col], df[y_col], p0=p0)
    y_pred = model_func(df[x_col], *params)
    return r2_score(df[y_col], y_pred)

r2_mcq = mcq_group_summary.groupby(['group_label', 'Amount'], observed=True).apply(
    calculate_r2, x_col='log_k', y_col='prop_delayed', model_func=logistic_growth, p0=[-4, 1]
).unstack()
print(r2_mcq)

print("\n--- Adj-Amt: R-squared for Group-Level Hyperboloid Fits ---")
r2_adj_amt = adj_amt_group_summary.groupby(['group_label', 'Amount']).apply(
    calculate_r2, x_col='Delay', y_col='mean_rsv', model_func=hyperboloid, p0=[0.1, 1]
).unstack()
print(r2_adj_amt)


# --- 2.2 Within-Procedure Reliability (Alternate-Forms Reliability) ---

# These analyses check if participants' discounting rates are consistent across
# different reward amounts within the same procedure.

print("\n\n--- MCQ: Within-Procedure Correlations (Number of Delayed Choices) ---")
mcq_reliability_pivot = mcq_participant_summary.pivot(
    index='ID', columns='Amount', values='num_delayed_choices'
)
print(mcq_reliability_pivot.corr())

print("\n--- Adj-Amt: Within-Procedure Correlations (AuC) ---")
adj_amt_reliability_pivot = adj_amt_participant_summary.pivot(
    index='ID', columns='Amount', values='auc'
)
# Rename columns for clarity in the correlation matrix
adj_amt_reliability_pivot.columns = [f"${c}" for c in adj_amt_reliability_pivot.columns]
print(adj_amt_reliability_pivot.corr())

## 3. Hypothesis Testing: Effects of Age on Discounting

This section replicates the core analyses from **Table 2** of the publication, which test the primary prediction of the **buffering hypothesis**. The key prediction is that **age will be associated with more patient decision-making (less discounting), but only for the low-income group**. No significant age effect is expected for the high-income group, as they are not experiencing the same level of financial scarcity.

To test this, we fit a series of Bayesian generalized linear mixed-effects models separately for each income group. The models predict discounting behavior as a function of age group.

In [ ]:
# --- 3.1 Model 1: Baseline Effect of Age ---

# This analysis tests the main prediction: is there an effect of age on discounting
# for each income group, before controlling for other factors?

# --- Low-Income Group Analysis ---
print("--- MODEL 1: AGE EFFECTS (LOW-INCOME GROUP) ---")
# Subset the data for the low-income group
low_income_mcq_df = mcq_model1_df[mcq_model1_df['income_group'] == 0]
low_income_adj_amt_df = adj_amt_model1_df[adj_amt_model1_df['income_group'] == 0]

# Fit Bayesian models
idata_mcq_age_low_inc = fit_bayesian_glmm(
    "num_delayed_choices ~ age_group_c",
    low_income_mcq_df,
    family='binomial'
)
idata_adj_amt_age_low_inc = fit_bayesian_glmm(
    "auc ~ age_group_c",
    low_income_adj_amt_df,
    family='beta'
)

# Display summaries
print("\nMCQ Results (Low-Income):")
print(summarize_pymc_model(idata_mcq_age_low_inc, var_names=['Intercept', 'age_group_c']))
print("\nAdj-Amt Results (Low-Income):")
print(summarize_pymc_model(idata_adj_amt_age_low_inc, var_names=['Intercept', 'age_group_c']))


# --- High-Income Group Analysis ---
print("\n\n--- MODEL 1: AGE EFFECTS (HIGH-INCOME GROUP) ---")
# Subset the data for the high-income group
high_income_mcq_df = mcq_model1_df[mcq_model1_df['income_group'] == 1]
high_income_adj_amt_df = adj_amt_model1_df[adj_amt_model1_df['income_group'] == 1]

# Fit Bayesian models
idata_mcq_age_high_inc = fit_bayesian_glmm(
    "num_delayed_choices ~ age_group_c",
    high_income_mcq_df,
    family='binomial'
)
idata_adj_amt_age_high_inc = fit_bayesian_glmm(
    "auc ~ age_group_c",
    high_income_adj_amt_df,
    family='beta'
)

# Display summaries
print("\nMCQ Results (High-Income):")
print(summarize_pymc_model(idata_mcq_age_high_inc, var_names=['Intercept', 'age_group_c']))
print("\nAdj-Amt Results (High-Income):")
print(summarize_pymc_model(idata_adj_amt_age_high_inc, var_names=['Intercept', 'age_group_c']))

## 4. Hypothesis Testing: Effects of Income on Discounting

This section replicates the complementary analyses from **Table 3** of the publication, providing a second test of the **buffering hypothesis**. The prediction here is the inverse of the previous section: **higher income will be associated with more patient decision-making, but only for the younger group**. Because older adults are already "buffered" by their greater emotional stability, their decision-making should be less influenced by income level.

To test this, we fit a series of Bayesian generalized linear mixed-effects models separately for each age group to examine the effect of income.

In [ ]:
# --- 4.1 Model 1: Baseline Effect of Income ---

# This analysis tests the complementary prediction: does higher income lead to
# less discounting, and is this effect present only for younger adults?

# --- Younger Adults Analysis ---
print("--- MODEL 1: INCOME EFFECTS (YOUNGER ADULTS) ---")
# Subset the data for the younger group
younger_mcq_df = mcq_model1_df[mcq_model1_df['age_group'] == 0]
younger_adj_amt_df = adj_amt_model1_df[adj_amt_model1_df['age_group'] == 0]

# Fit Bayesian models
idata_mcq_income_younger = fit_bayesian_glmm(
    "num_delayed_choices ~ income_group_c",
    younger_mcq_df,
    family='binomial'
)
idata_adj_amt_income_younger = fit_bayesian_glmm(
    "auc ~ income_group_c",
    younger_adj_amt_df,
    family='beta'
)

# Display summaries
print("\nMCQ Results (Younger Adults):")
print(summarize_pymc_model(idata_mcq_income_younger, var_names=['Intercept', 'income_group_c']))
print("\nAdj-Amt Results (Younger Adults):")
print(summarize_pymc_model(idata_adj_amt_income_younger, var_names=['Intercept', 'income_group_c']))


# --- Older Adults Analysis ---
print("\n\n--- MODEL 1: INCOME EFFECTS (OLDER ADULTS) ---")
# Subset the data for the older group
older_mcq_df = mcq_model1_df[mcq_model1_df['age_group'] == 1]
older_adj_amt_df = adj_amt_model1_df[adj_amt_model1_df['age_group'] == 1]

# Fit Bayesian models
idata_mcq_income_older = fit_bayesian_glmm(
    "num_delayed_choices ~ income_group_c",
    older_mcq_df,
    family='binomial'
)
idata_adj_amt_income_older = fit_bayesian_glmm(
    "auc ~ income_group_c",
    older_adj_amt_df,
    family='beta'
)

# Display summaries
print("\nMCQ Results (Older Adults):")
print(summarize_pymc_model(idata_mcq_income_older, var_names=['Intercept', 'income_group_c']))
print("\nAdj-Amt Results (Older Adults):")
print(summarize_pymc_model(idata_adj_amt_income_older, var_names=['Intercept', 'income_group_c']))

## 5. Composite Score Analysis

The final analyses use the composite z-score—a single, robust measure that combines data from both the MCQ and Adj-Amt procedures—to provide a holistic test of the buffering hypothesis.

First, we replicate the focused correlation tests from the paper. As predicted, we expect to find a significant positive correlation between age and financial patience (higher z-score) **only in the low-income group**. Conversely, we expect a significant positive correlation between income and financial patience **only in the younger group**.

In [ ]:
# --- 5.1 Focused Correlation Tests with Composite Score ---

# Use the composite z-score to perform the two key hypothesis tests in a 
# single, tidy analysis for each hypothesis.

# Test 1: Correlation between Age and Discounting within each Income Group
print("--- Correlation of Age Group with Discounting z-score ---")
age_correlations = composite_score_df.groupby('income_group').apply(
    lambda df: pearsonr(df['mean_z_score'], df['age_group']),
    include_groups=False
).apply(pd.Series)

age_correlations.columns = ['correlation_r', 'p_value']
age_correlations.index = age_correlations.index.map({0: 'Low-Income', 1: 'High-Income'})
print(age_correlations)


# Test 2: Correlation between Income and Discounting within each Age Group
print("\n\n--- Correlation of Income Group with Discounting z-score ---")
income_correlations = composite_score_df.groupby('age_group').apply(
    lambda df: pearsonr(df['mean_z_score'], df['income_group']),
    include_groups=False
).apply(pd.Series)

income_correlations.columns = ['correlation_r', 'p_value']
income_correlations.index = income_correlations.index.map({0: 'Younger', 1: 'Older'})
print(income_correlations)

## 6. Quantifying the Magnitude of the Age Difference

Having established the predicted pattern of effects, these final analyses use the composite z-score to quantify the magnitude of the buffering effect. By fitting a series of linear regression models, we can estimate the practical size of the age-related increase in financial patience specifically for the low-income group. This corresponds to the final results presented in the "Discussion" section of the publication.

In [ ]:
# --- 6.1 Group Means and Interaction Model ---

# First, calculate and display the mean composite z-score for each of the four
# experimental groups to provide context for the regression models.
print("--- Mean Composite z-Score by Group ---")
mean_z_scores = composite_score_df.groupby(['age_group', 'income_group'])['mean_z_score'].mean().reset_index()
mean_z_scores['Age_Group'] = np.where(mean_z_scores['age_group'] == 0, "Younger", "Older")
mean_z_scores['Income_Group'] = np.where(mean_z_scores['income_group'] == 0, "Low-Income", "High-Income")
print(mean_z_scores[['Age_Group', 'Income_Group', 'mean_z_score']])

# Next, fit a full linear model with an interaction term. This formally tests
# whether the effect of age on discounting depends on a person's income level.
print("\n\n--- Full Model: z_score ~ Age_Group * Income_Group ---")
full_interaction_model = smf.ols(
    'mean_z_score ~ age_group_c * income_group_c', 
    data=composite_score_df
).fit()
print(full_interaction_model.summary())

# --- 6.2 Estimating the Continuous Age Effect by Income Group ---

# Finally, model the effect of age as a continuous variable within each
# income group separately. This allows us to estimate the year-over-year
# change in financial patience (z-score).

print("\n\n--- Continuous Age Effect within Low-Income Group ---")
# As predicted by the buffering hypothesis, we expect a significant, positive slope for age.
low_income_age_model = smf.ols(
    'mean_z_score ~ Age', 
    data=composite_score_df[composite_score_df['income_group'] == 0]
).fit()
print(low_income_age_model.summary())

print("\n\n--- Continuous Age Effect within High-Income Group ---")
# In the high-income group, we expect the slope for age to be non-significant.
high_income_age_model = smf.ols(
    'mean_z_score ~ Age', 
    data=composite_score_df[composite_score_df['income_group'] == 1]
).fit()
print(high_income_age_model.summary())